In [4]:
import boto3
import yaml
import sqlite3
import streamlit as st

AWS_ACCESS_KEY = st.secrets["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = st.secrets["AWS_SECRET_KEY"]

bucket = "noaa-goes18"

In [ ]:
year_list = []
year_day_list = []
year_day_hour_set = set()

client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

prefix = "ABI-L1b-RadC/"
result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
for o in result.get("CommonPrefixes"):
    year_list.append(o.get("Prefix"))

for year in year_list:
    prefix = year
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
    for o in result.get("CommonPrefixes"):
        year_day_list.append(o.get("Prefix"))

for year_day in year_day_list:
    prefix = year_day
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
    for o in result.get("CommonPrefixes"):
        meta_data = o.get("Prefix").split("/")
        year_day_hour_set.add((meta_data[1], meta_data[2], meta_data[3]))

conn = sqlite3.connect("noaa_goes_date.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS noaa_goes_date")
# Create a new table
cursor.execute(
    "CREATE TABLE IF NOT EXISTS noaa_goes_date (year text, day text, hour text)"
)
for date in year_day_hour_set:
    year, day, hour = date
    cursor.execute("INSERT INTO noaa_goes_date VALUES (?, ?, ?)", (year, day, hour))
conn.commit()
conn.close()